In [7]:
from functools import reduce, partial
product = partial(reduce, lambda x,y: x * y)

import jax.numpy as jnp
from jax import random, vmap, grad

import arviz as az
from matplotlib import pyplot as plt
from ipywidgets.widgets import FloatSlider
from ipywidgets import interact
from scipy.interpolate import BSpline
from scipy.stats import gaussian_kde, zscore

import numpyro
import numpyro.distributions as dist
import numpyro.optim as optim
from numpyro.diagnostics import hpdi, print_summary
from numpyro.infer import Predictive, SVI, Trace_ELBO, init_to_value, NUTS, MCMC
from numpyro.infer.autoguide import AutoLaplaceApproximation, AutoNormal, AutoDiagonalNormal

import xgboost as xgb

import pandas as pd

from clean import Clean

key = random.PRNGKey(0)

In [3]:
cleanB2 = Clean("data/b2-10262021.csv", abso=False, unknown_feature="B2")
dfB2 = cleanB2.df
dfB2.head()

,A,D,B0,B1,B2,Root,Y,newton_delta,squared_newton_delta
0,3.006537,4.521234e+11,1.877143e+11,3.682367e+09,4.616150e+10,-8.184270e+10,-1.204838e-08,1.280042e+11,1.638507e+22
1,2.252032,3.242883e+11,1.049468e+11,1.266971e+11,1.270974e+11,9.277215e+10,-1.812794e-07,3.432522e+10,1.178221e+21
2,1.047534,1.905252e+11,3.068875e+11,9.588425e+10,1.370219e+11,2.674530e+08,-4.008832e-08,1.367544e+11,1.870177e+22
3,4.034365,5.000290e+10,1.776733e+11,2.439855e+11,1.828785e+11,1.317694e+05,-3.244612e-05,1.828784e+11,3.344451e+22
4,1.396830,2.371336e+11,1.062002e+10,1.470824e+10,1.025206e+11,-7.170986e+10,-6.604643e-09,1.742305e+11,3.035627e+22


(1521, 9)


In [10]:

def dimension1_invariant_D(A: float, D: float, *balances: list) -> float: 
    """invariant divided by product of balances"""
    n = len(balances)
    n_raised_2n = n ** (2 * n)
    return (
        product(D / b for b in balances) * D
        +
        (A - n ** (- n)) * n_raised_2n * D
        + 
        (- A * n_raised_2n * sum(balances))
    )

def dimensionless_invariant_D(A: float, D: float, *balances: list) -> float: 
    """invariant divided by product of balances divided by D"""
    n = len(balances)
    n_raised_2n = n ** (2 * n)
    return (
        product(D / b for b in balances)
        +
        (A - 1 / n ** n) * n_raised_2n
        + 
        (- A * n_raised_2n * sum(balances) / D)
    )

def invariant_B2(A: float, D: float, b0: float, b1: float, b2: float) -> float: 
    """ """
    n = 3 
    return (
        b2 ** 2 
        + 
        (b0 + b1 + (1 / A / n ** n - 1) * D) * b2
        + 
        (- D ** (n + 1)) / A / n ** (2 * n) / b0 / b1
    )

def dimensionless_invariant_B2(A: float, D: float, b0: float, b1: float, b2: float) -> float: 
    """Divide by D ^ 2"""
    n = 3
    return (
        (b2 / D) ** 2
        + 
        ((b0 + b1) / D ** 2 + (1 / A / n ** n - 1) / D) * b2
        + 
        (- D ** (n + 1 - 2)) / A / n ** (2 * n) / b0 / b1
    )


In [20]:
dfB2 = dfB2.assign(
    invariant_root=invariant_B2(dfB2.A, dfB2.D, dfB2.B0, dfB2.B1, dfB2.Root), 
    dimensionless_invariant_root=dimensionless_invariant_B2(dfB2.A, dfB2.D, dfB2.B0, dfB2.B1, dfB2.Root),
    invariant=invariant_B2(dfB2.A, dfB2.D, dfB2.B0, dfB2.B1, dfB2.B2), 
    dimensionless_invariant=dimensionless_invariant_B2(dfB2.A, dfB2.D, dfB2.B0, dfB2.B1, dfB2.B2)
)
(dfB2[["invariant_root", "dimensionless_invariant_root"]] < 1e-7).sum()

invariant_root                   5756
dimensionless_invariant_root    10000
dtype: int64

In [21]:
negroot = dfB2[dfB2.Root < 0]
print(negroot.shape)
(negroot[["invariant_root", "dimensionless_invariant_root"]] < 1e-7).sum()

(1521, 13)


invariant_root                   984
dimensionless_invariant_root    1521
dtype: int64

In [22]:
dfB2.min()

A                               3.463852e-04
D                               1.233141e+06
B0                              1.520559e+07
B1                              2.343430e+06
B2                              1.033991e+07
Root                           -1.144398e+13
Y                              -3.587333e-01
newton_delta                   -2.120166e+13
squared_newton_delta            1.742624e+14
invariant                      -4.505295e+26
dimensionless_invariant        -4.200342e+03
invariant_root                 -1.030792e+11
dimensionless_invariant_root   -9.094947e-13
dtype: float64

In [6]:
dat = dfB2 # 

def model(D, B0, B1, B2, Root=None):
    """Get the posterior distribution of Root values.
    """
    LEN_BALANCES = 3
    A = 85

    numpyro.sample("root", dist.Normal(Root.mean(), Root.std()))
    sigma = numpyro.sample("sigma", dist.Uniform(0, I.std()))

    numpyro.sample("inv_D", dist.Normal(I, sigma), obs=inv_D)


quap = AutoLaplaceApproximation(model)
svi = SVI(
    model,
    quap,
    optim.Adam(1),
    Trace_ELBO(),
    D=df_z.D.values,
    inv_D=df_z.Y.values,
)
svi_result = svi.run(random.PRNGKey(0), 2000)
params_svi_result = svi_result.params

samples = quap.sample_posterior(random.PRNGKey(1), params_svi_result, (1000,))
print_summary({key: value for key, value in samples.items() if key != "I"}, 0.89, False)